In [ ]:
# For Google Colab to access Drive
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# David Cabezas Berrido

# p2pGAN-discriminator.ipynb

# Shows examples of how discriminator works

import tensorflow as tf
import matplotlib.pyplot as plt

PATH = '/content/drive/My Drive/TFG-Image-Optimization' # for Google Colab
INPATH = PATH+'/discriminator-inputs/' # The discriminator examples are mixed images (half real, half generated) of shape 1536 x 1024
DARK = INPATH+'/dark/' # Dark images of the appropriate shape
MODELS = PATH+'/models/'
OUPATH = PATH+'/results/discriminator/' # for Google Colab

urls=['fuji-20018-x8.jpg','fuji-20069-x6.jpg','fuji-20167-x2.jpg','fuji-20167-x16.jpg']

In [ ]:
# Load discriminator
discriminator = tf.keras.models.load_model(MODELS+'GAN-discriminator')

In [ ]:
# Function to read images
def decode_normalize(path):
    inimg = tf.cast(tf.image.decode_jpeg(tf.io.read_file(path)),tf.float32)[...,:3]
    inimg = inimg/127.5-1
    return inimg

In [ ]:
for url in urls: # Needs to load both dark and generated/real images
    path = INPATH+url
    dat = tf.data.Dataset.from_tensor_slices([path])
    dat = dat.map(decode_normalize)
    dat = dat.batch(1)
    dark_path=DARK+url
    dark_dat = tf.data.Dataset.from_tensor_slices([dark_path])
    dark_dat = dark_dat.map(decode_normalize)
    dark_dat = dark_dat.batch(1)
    for img in dat:
        for dark_img in dark_dat:
            prediction = discriminator([dark_img,img], training=True) # Prediction
            out=prediction[0,...,-1]*0.5+0.5
            plt.imshow(out, cmap='RdBu_r', vmin=-4, vmax=8)
            plt.colorbar()
            plt.savefig(OUPATH+url)
            plt.show()
            #plt.imsave(OUPATH+url, out, cmap='RdBu_r', vmin=-4, vmax=8)